# X-Ray Absorption Spectra Using BSE
**by <span style="color:darkgreen">Christian Vorwerk</span>, <span style="color:darkgreen">Caterina Cocchi</span>, & <span style="color:darkgreen">Keith Gilmore</span> for [<span style="color:DarkGoldenRod">exciting *oxygen*</span>](http://exciting.wikidot.com/oxygen)**

<font size="2">(Jupyter notebook by <span style="color:darkgreen">Megha Arya</span>)</font>
<hr style="border:2px solid #DDD"> </hr>

**<span style="color:firebrick">Purpose</span>**: In this tutorial, we present an example of the calculation of the **X-ray** absorption near-edge structure (**XANES**) spectra by solving the Bethe-Salpeter equation (**BSE**). For this purpose, we show the examples of the N **K**-edge in cubic boron-nitride, as well as of the Ti **L<sub>2,3</sub>**-edge in rutile titanium dioxide.

<hr style="border:2px solid #DDD"> </hr>

<div class="alert alert-block alert-warning">
    
**Table of Contents**

[0. Preparation](#0)
    
[1. Introduction](#1)
    
[2. The N-K Edge in Cubic Boron Nitride](#2)
    
  - Exercise 1
    
[3. The Ti L2,3-Edge in Rutile TiO2](#3)
    
  - Preparation of the Input File and Ground-State Calculation
  - Calculation of XANES Spectra and Core-Exciton Output
  - Exercise 2

</div>

<a id='0'></a>
<hr style="border:1px solid #DDD"> </hr>

### <span style="color:#15317E">0. Preparation</span>

**<span style="color:firebrick">Read the following paragraph before starting with the rest of this tutorial!</span>**

Before running any Jupyter tutorials, please refer to the **`00_before_starting.md`** document on how to correctly set up the environment. This only needs to be done once. After which, the **venv** can be (re)activated from **exciting**'s root directory:

<div style="background-color: rgb(224, 224, 224);">

```bash
source tools/excitingjupyter/venv/excitingvenv/bin/activate
```

</div>

Determine the root directory of **exciting**, in which the binary executables as well as the species files are located by executing the following cell.

In [1]:
import os
from os import getcwd
from os.path import join
from excitingjupyter.utilities import get_exciting_root

exciting_root = get_exciting_root()
exciting_species = join(exciting_root, "species")
print(exciting_root)

/users/tfk/amegha/exciting


<a id='1'></a>
<hr style="border:1px solid #DDD"> </hr>

### <span style="color:#15317E">1. Introduction</span>

The approach to calculate core spectra from the solution of the Bethe-Salpeter equation (**BSE**) in **exciting** is very similar to the approach to optical spectra described in the tutorial [<span style="color:firebrick">Excited states from BSE</span>](http://exciting.wikidot.com/oxygen-excited-states-from-bse). The main difference resides in the initial states of the transitions, which are **core states**. As such, they are solutions of the radial Dirac equation in the muffin-tin spheres.

A description of the basic setup for **BSE** calculations can be found in [<span style="color:firebrick">Excited states from BSE</span>](http://exciting.wikidot.com/oxygen-excited-states-from-bse). In addition, all the input parameters are described in [<span style="color:firebrick">Input Reference</span>](http://exciting.wikidot.com/ref:input). Note that, in order to obtain reliable **X-ray** absorption spectra, the calculation needs to be converged with respect to several parameters, such as the size of the **k**-mesh.

<a id='2'></a>
<hr style="border:1px solid #DDD"> </hr>

### <span style="color:#15317E">2. The N-K Edge in Cubic Boron Nitride</span>

Here, we consider the N **K**-edge in c-BN. As a first step, create a new directory, which you can call, *e.g.*, **BN-XANES** and move inside it.

As for all other tutorials, you can choose between executing all steps manually from your command line, or to simply execute the python cells in this Jupyter notebook.

In [2]:
%%bash
# Create working directory
mkdir -p BN-XANES

In [3]:
cwd_BN = join(getcwd(), 'BN-XANES')

We start with a ground-state calculation of c-BN. Below is an example of the corresponding input file.

```xml
<input>
 
   <title>Cubic boron nitride: Ground-State Calculation</title>
 
   <structure autormt="true" speciespath="$EXCITINGROOT/species">
      <crystal scale="6.83136">
         <basevect>0.5 0.0 0.5</basevect>
         <basevect>0.0 0.5 0.5</basevect>
         <basevect>0.5 0.5 0.0</basevect>
      </crystal>
      <species speciesfile="B.xml" >
         <atom coord="0.00 0.00 0.00" />
      </species>
      <species speciesfile="N.xml">
         <atom coord="0.25 0.25 0.25" />
      </species>
   </structure>
 
   <groundstate 
      ngridk="4 4 4"
      xctype="GGA_PBE_SOL"
      gmaxvr="14.0" />
 
</input>

```

For manual execution, copy and paste the above into the new file **input.xml**. For an execution within the notebook, execute the following cells. 

In [4]:
# Writing the input as an XML file
from excitingjupyter.utilities import re_input

notebook = join(os.getcwd(), 'xray_absorption_spectra_using_bse.ipynb')
input_str = re_input(notebook, "Cubic boron nitride: Ground-State Calculation")
with open(join(cwd_BN, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [5]:
# Replacing the string "$EXCITINGROOT/species" in input.xml by the actual value of the environment variable
with open(join(cwd_BN, 'input.xml'), "r+") as fin:
    file_content = fin.read()
    file_content = file_content.replace('$EXCITINGROOT/species', f'{exciting_species}')
    fin.seek(0)
    fin.write(file_content)

Run the ground-state calculation with the usual command i.e. in order to run **exciting**, you simply need to execute the **exciting_smp** binary in the running directory. After a few seconds, the calculation should be finished.

Here we use the <code>time</code> command before **exciting_smp** in order to get, at the end of the run, the elapsed time explicitly written on the screen.

In [6]:
%%bash
# Move into the run directory
cd BN-XANES
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

 ### Using specified input file: input.xml


   Elapsed time = 0m6s


After the ground-state calculation is completed, we inspect the output file **EVALCORE.OUT**, which contains the information about core electrons. Since we are interested here in the excitations from the Nitrogen **K**-edge, we have to look specifically for the N **1*s*** electron. For this purpose we have to consider the core state with quantum numbers ***n*=1** and ***l*=0**:

<div style="background-color: rgb(224, 224, 224);">
    
```bash
Species :    2 (N), atom :    1
 n =  1, l =  0, k =  1 :   -13.38038902
```
</div>

This can be checked within the notebook by executing the following cell.

In [ ]:
!tail -n 2 BN-XANES/EVALCORE.OUT

From the file **EVALCORE.OUT** we obtain the information that the N **1*s*** electron in cubic BN has an energy of about -13.38 Ha. With this knowledge, we can add to the input file the parameters required to perform a **XANES** calculation. Since we are dealing with excited-state properties, we have to insert the <code><span style="color:green">xs</span></code> element inside the <code><span style="color:green">input</span></code> element. Do not forget to skip the ground-state run by inserting the attribute <code><span style="color:mediumblue">do</span>=<span style="color:firebrick">"skip"</span></code> inside the <code><span style="color:green">groundstate</span></code> element. Now the file **input.xml** will look like the following.

```xml
<input>
 
   <title>Cubic boron nitride: Excited-State Calculation</title>
 
   <structure autormt="true" speciespath="$EXCITINGROOT/species">
      <crystal scale="6.83136">
         <basevect>0.5 0.0 0.5</basevect>
         <basevect>0.0 0.5 0.5</basevect>
         <basevect>0.5 0.5 0.0</basevect>
      </crystal>
      <species speciesfile="B.xml" >
         <atom coord="0.00 0.00 0.00" />
      </species>
      <species speciesfile="N.xml">
         <atom coord="0.25 0.25 0.25" />
      </species>
   </structure>
 
   <groundstate 
      do="skip"
      ngridk="4 4 4"
      xctype="GGA_PBE_SOL"
      gmaxvr="14.0" />
      
   <xs
      xstype="BSE" 
      ngridk="2 2 2" 
      vkloff="0.05 0.15 0.25"
      ngridq="2 2 2"
      nempty="30" 
      gqmax="3.0" 
      broad="0.018"
      tevout="true" >
 
      <energywindow 
         intv="13.4 16.5" 
         points="1500" />
 
      <screening 
         screentype="full" 
         nempty="100" />
 
      <BSE 
         xas="true"
         xasspecies="2"
         xasatom="1"
         xasedge="K"
         bsetype="singlet" 
         nstlxas="1 15" />
 
      <qpointset>
         <qpoint>0.0 0.0 0.0</qpoint>
      </qpointset>
 
   </xs> 
 
</input>

```

For manual execution, update the **input.xml** file on terminal. For running the calculation in the notebook, execute the following cells.

In [ ]:
# Writing the input as an XML file
from excitingjupyter.utilities import re_input

notebook = join(os.getcwd(), 'xray_absorption_spectra_using_bse.ipynb')
input_str = re_input(notebook, "Cubic boron nitride: Excited-State Calculation")
with open(join(cwd_BN, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [ ]:
# Replacing the string "$EXCITINGROOT/species" in input.xml by the actual value of the environment variable
with open(join(cwd_BN, 'input.xml'), "r+") as fin:
    file_content = fin.read()
    file_content = file_content.replace('$EXCITINGROOT/species', f'{exciting_species}')
    fin.seek(0)
    fin.write(file_content)

You should be already familiar with many parameters shown above. However, it is useful to highlight a few points here:

- The element <code><span style="color:green">energywindow</span></code> is now chosen from 13.4 to 16.5 Ha. This is consistent with the energy of the N 1s state, which represents the investigated absorption edge.
- The attribute <code><span style="color:mediumblue">xas</span>=<span style="color:firebrick">"true"</span></code> in the <code><span style="color:green">BSE</span></code> element triggers the core-level calculation.
- The attributes <code><span style="color:mediumblue">xasspecies</span></code>, <code><span style="color:mediumblue">xasatom</span></code>, and <code><span style="color:mediumblue">xasedge</span></code> uniquely define the initial states of the transition. In our case, we choose <code><span style="color:mediumblue">xasspecies</span>=<span style="color:firebrick">"2"</span></code> to select Nitrogen, <code><span style="color:mediumblue">xasatom</span>=<span style="color:firebrick">"1"</span></code> to identify the one and only N atom in the unit cell, and the <code><span style="color:mediumblue">xasedge</span>=<span style="color:firebrick">"K"</span></code> to specify that we are performing a **K**-edge calculation.
- The attribute <code><span style="color:mediumblue">nstlxas</span></code> describes the number of unoccupied states used in the **BSE**-Hamiltonian counting from the lowest unoccupied band. Setting it to <code><span style="color:mediumblue">nstlxas</span>=<span style="color:firebrick">"1 15"</span></code>, we use the lowest 15 unoccupied bands in the calculation.

We are now ready to run the calculation with the usual command. 

In [ ]:
%%bash
# Move into the run directory
cd BN-XANES 
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

The calculation will run for a few seconds. You can follow the progress on the shell, as described in the Tutorial [<span style="color:firebrick">Excited states from BSE</span>](http://exciting.wikidot.com/oxygen-excited-states-from-bse). The produced output is organized as in **BSE** calculations run in the optical region (see [<span style="color:firebrick">Excited states from BSE</span>](http://exciting.wikidot.com/oxygen-excited-states-from-bse)). A number of files are printed in the working directory. Among them, the file **INFOXS.OUT** shows the general messages for the calculation procedure. The physically relevant output quantities are included in the sub-folders named **<span style="color:mediumblue">EPSILON</span>**, **<span style="color:mediumblue">EXCITON</span>**, and **<span style="color:mediumblue">LOSS</span>**. Since we are interested in the X-ray absorption spectrum of cubic-BN, given by the imaginary part of the dielectric function, we look at the results contained in the **<span style="color:mediumblue">EPSILON</span>** folder.

In order to plot the spectrum from your command line, execute the following commands:

<div style="background-color: rgb(224, 224, 224);">
    
```bash
cp EPSILON/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT singlet-TDA
PLOT-files.py -f singlet-TDA  -lx 'Energy [eV]'  -ly 'Im $\varepsilon_M$'  -t 'Macroscopic dielectric function'  -g  -rc  -cy 3  -nl
```

</div>    
    
The result is stored in **PLOT.png** and **PLOT.eps** and should resemble this plot:

<div>
<img src="images_xray_absorption_spectra_using_bse/graph1.jpg" width="500" align="left"/>
</div>

The plot can also be produced within this notebook by executing the following cell.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (7, 5))

from excitingtools import parser_chooser
epsilon_results_BN: dict = parser_chooser(f'{cwd_BN}/EPSILON/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

plt.plot(epsilon_results_BN['frequency'], epsilon_results_BN['imag_oscillator_strength'], color = 'mediumblue')
plt.xlabel('Energy [eV]', fontsize=17)
plt.ylabel(r"$\mathrm{Im}\;\; \varepsilon_M$", fontsize = 17)
plt.title('Macroscopic dielectric function', fontsize = 20)
plt.xlim(360, 450)
plt.ylim(0.000, 0.025)
plt.grid()

plt.show()

As all the main output files can be parsed by Python, in the example below, the results contained in main output file INFO.OUT are saved in the dictionary **converged_results_BN** and the results obtained for the loss function are saved in the dictionary **loss_results_BN**. The results obtained for the dielectric function are saved in the dictionary **epsilon_results_BN** as already defined above.

In [ ]:
results = parser_chooser(join(cwd_BN, 'INFO.OUT'))
max_scf = max([int(i) for i in results['scl'].keys()])
assert max_scf <= 12, "Expect max 12 SCF iterations to converge"
converged_results_BN: dict = results['scl'][str(max_scf)]

loss_results_BN: dict = parser_chooser(f'{cwd_BN}/LOSS/LOSS_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

We can further check that the obtained results are correct by comparing them with the reference calculations. To do so within the notebook, execute the following cell :

In [ ]:
from assertions_xray_absorption_bse import *

# Test results in main output file INFO.OUT 
test_groundstate_BN(converged_results_BN)

# Test results of dielectric function calculations
test_dielectric_BN(epsilon_results_BN, reference = 'reference_xray_absorption_bse/BN/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

# Test results of loss function calculations
test_loss_BN(loss_results_BN, reference = 'reference_xray_absorption_bse/BN/LOSS_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

**<span style="color:green">Exercise 1</span>**

- Calculate the N **K**-edge spectra in the random-phase approximation (**RPA**) by setting the parameter <code><span style="color:mediumblue">bsetype</span>=<span style="color:firebrick">"RPA"</span></code>. Compare the spectra of the **BSE** and the **RPA** calculation.
- Calculate the boron **K**-edge in c-BN.

<a id='3'></a>
<hr style="border:1px solid #DDD"> </hr>

### <span style="color:#15317E">3. The Ti L<sub>2,3</sub>-Edge in Rutile TiO<sub>2</sub></span>

**i) Preparation of the Input File and Ground-State Calculation**

As the next example, we will calculate the Ti **L<sub>2,3</sub>**-edge XANES of the rutile phase of TiO2. Before starting the new calculation, go back to the parent directory, create a directory named **TiO2-XANES**, and move into it. 

In [ ]:
%%bash
# Create working directory
mkdir -p TiO2-XANES

In [ ]:
cwd_TiO2 = join(getcwd(), 'TiO2-XANES')

In the standard notation, exciting the Ti **L<sub>2,3</sub>**-edge in rutile TiO2 corresponds to considering transitions from all the Ti ***2p*** states. First, we need to make sure that the initial states are treated as core states in **exciting**. 

Therefore, we look at the species file $EXCITINGROOT/species/Ti.xml.

In [ ]:
!cat {exciting_species}/Ti.xml

You will see on the screen a list of atomic states. We are interested in the first few lines. The 2*p*-electrons, given by

<div style="background-color: rgb(224, 224, 224);">

```bash
...
    <atomicState n="2" l="1" kappa="1" occ="2.00000" core="true"/>
    <atomicState n="2" l="1" kappa="2" occ="4.00000" core="true"/>
...
```
    
</div>

are indeed treated as core states. In fact, in both lines <code><span style="color:green">atomicState</span></code>, the attribute <code><span style="color:mediumblue">core</span></code> is set to **"true"**. The two different atomic states correspond to the Ti 2*p*<sub>1/2</sub> and Ti 2*p*<sub>3/2</sub> states, respectively.

The file below is an example input file for performing the ground-state calculation on rutile TiO<sub>2</sub>.

```xml
<input>
 
   <title>Rutile TiO2: Ground-State Calculation</title>
 
   <structure speciespath="$EXCITINGROOT/species">
      <crystal>
         <basevect>   8.763555397       0.000000000       0.000000000</basevect>
         <basevect>   0.000000000       8.763555397       0.000000000</basevect>
         <basevect>   0.000000000       0.000000000       5.610015465</basevect>
      </crystal>
      <species speciesfile="Ti.xml" rmt="1.8000">
         <atom coord="0.0000000000      0.0000000000      0.0000000000"/>
         <atom coord="0.5000000000      0.5000000000      0.5000000000"/>
      </species>
      <species speciesfile="O.xml" rmt="1.8000">
         <atom coord="0.3050853616      0.3050853616      0.0000000000"/>
         <atom coord="0.6949146384      0.6949146384      0.0000000000"/>
         <atom coord="0.1949146384      0.8050853616      0.5000000000"/>
         <atom coord="0.8050853616      0.1949146384      0.5000000000"/>
      </species>
   </structure>
 
   <groundstate 
      rgkmax="7.0"
      ngridk="4 4 6"
      xctype="GGA_PBE_SOL"
      epsocc="1.0d-6">
   </groundstate>
 
</input>

```

On terminal, copy it into the file input.xml. For an execution within the notebook, execute the following cells.

In [ ]:
# Writing the input as an XML file
from excitingjupyter.utilities import re_input

notebook = join(os.getcwd(), 'xray_absorption_spectra_using_bse.ipynb')
input_str = re_input(notebook, "Rutile TiO2: Ground-State Calculation")
with open(join(cwd_TiO2, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [ ]:
# Replacing the string "$EXCITINGROOT/species" in input.xml by the actual value of the environment variable
with open(join(cwd_TiO2, 'input.xml'), "r+") as fin:
    file_content = fin.read()
    file_content = file_content.replace('$EXCITINGROOT/species', f'{exciting_species}')
    fin.seek(0)
    fin.write(file_content)

We can now  run the calculation with the usual command. 

In [ ]:
%%bash
# Move into the run directory
cd TiO2-XANES
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

Once the calculation is finished, we can identify the Ti ***2p*** states in the output file **EVALCORE.OUT**, and with them determine the <code><span style="color:green">energywindow</span></code> for the **XANES** calculation. To do so, we open EVALCORE.OUT and look for the following lines:

<div style="background-color: rgb(224, 224, 224);">

```bash
Species :    1 (Ti), atom :    1
 n =  1, l =  0, k =  1 :   -178.2875257    
 n =  2, l =  0, k =  1 :   -19.33482231    
 n =  2, l =  1, k =  1 :   -16.08151430    
 n =  2, l =  1, k =  2 :   -15.86989435
```   
</div>

This can be checked within the notebook by executing the following cell.

In [ ]:
!head -n 6 TiO2-XANES/EVALCORE.OUT

The core levels we are interested in are ***2p*** states, which have principal quantum number ***n=2***, and angular quantum number ***l=1***. $\kappa$ is a unique combination of the quantum numbers ***l*** and ***J*** given by

$$\kappa = \begin{equation}
\left\{ 
  \begin{aligned}
    -l-1\,for\,J=l+\frac{1}{2}\\
    l\,for\,J=l-\frac{1}{2}  \\
  \end{aligned}
  \right.
\end{equation}
$$

The Ti **2*p<sub>1/2</sub>*** states ($\kappa$=**1**) are at about -16.1 Ha and the Ti **2*p<sub>3/2</sub>*** ones ($\kappa$=**-2**) are at about -15.87 Ha. Thus, we define the energy window for our XANES by setting <code><span style="color:green">energywindow</span></code> = **"15.0 18.0"**.

**ii) Calculation of XANES Spectra and Core-Exciton Output**

Now we can start the XANES calculation by adding the <code><span style="color:green">xs</span></code> element to the input file. Remember to set the attribute <code><span style="color:mediumblue">do</span>=<span style="color:firebrick">"skip"</span></code> inside the element <code><span style="color:green">groundstate</span></code> to skip the ground-state calculation. Now the file **input.xml** will look like the following.

```xml
<input>
 
   <title>Rutile TiO2: Excited-State Calculation</title>
 
   <structure speciespath="$EXCITINGROOT/species">
      <crystal>
         <basevect>   8.763555397       0.000000000       0.000000000</basevect>
         <basevect>   0.000000000       8.763555397       0.000000000</basevect>
         <basevect>   0.000000000       0.000000000       5.610015465</basevect>
      </crystal>
      <species speciesfile="Ti.xml" rmt="1.8000">
         <atom coord="0.0000000000      0.0000000000      0.0000000000"/>
         <atom coord="0.5000000000      0.5000000000      0.5000000000"/>
      </species>
      <species speciesfile="O.xml" rmt="1.8000">
         <atom coord="0.3050853616      0.3050853616      0.0000000000"/>
         <atom coord="0.6949146384      0.6949146384      0.0000000000"/>
         <atom coord="0.1949146384      0.8050853616      0.5000000000"/>
         <atom coord="0.8050853616      0.1949146384      0.5000000000"/>
      </species>
   </structure>
 
   <groundstate 
      do = "skip"
      rgkmax="7.0"
      ngridk="4 4 6"
      xctype="GGA_PBE_SOL"
      epsocc="1.0d-6">
   </groundstate>
   
   <xs
      xstype="BSE" 
      ngridk="2 2 3"
      ngridq="2 2 3"
      rgkmax="7.0" 
      vkloff="0.05 0.15 0.25"
      nempty="30" 
      gqmax="1.0" 
      broad="0.018"
      tevout="true" >
 
      <energywindow 
         intv="15.0 18.0" 
         points="1500" />
      <screening 
         screentype="full" 
         nempty="75" />
      <BSE
         xas="true"
         xasspecies="1"
         xasatom="2"
         xasedge="L23" 
         bsetype="singlet"  
         nstlxas="1 20" />
      <qpointset>
         <qpoint>0.0 0.0 0.0</qpoint>
      </qpointset>
   </xs>
 
</input>

```

For manual execution, update the **input.xml** file on terminal. For running the calculation in the notebook, execute the following cells.

In [ ]:
# Writing the input as an XML file
from excitingjupyter.utilities import re_input

notebook = join(os.getcwd(), 'xray_absorption_spectra_using_bse.ipynb')
input_str = re_input(notebook, "Rutile TiO2: Excited-State Calculation")
with open(join(cwd_TiO2, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [ ]:
# Replacing the string "$EXCITINGROOT/species" in input.xml by the actual value of the environment variable
with open(join(cwd_TiO2, 'input.xml'), "r+") as fin:
    file_content = fin.read()
    file_content = file_content.replace('$EXCITINGROOT/species', f'{exciting_species}')
    fin.seek(0)
    fin.write(file_content)

We can now again run the calculation with the usual command. 

In [ ]:
%%bash
# Move into the run directory
cd TiO2-XANES 
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

Once the calculation is finished (it will take slightly longer than the one on c-BN above), we can inspect the computed **XANES** spectra. To do so, as usual, we consider the imaginary part of the dielectric function, which is included in the **<span style="color:mediumblue">EPSILON</span>** folder. Type the following commands in order to plot the spectrum from your command line:

<div style="background-color: rgb(224, 224, 224);">
    
```bash
cp EPSILON/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT singlet-TDA
PLOT-files.py -f singlet-TDA  -lx 'Energy [eV]'  -ly 'Im $\varepsilon_M$'  -t 'Macroscopic dielectric function'  -x 420 470  -g  -rc  -cy 3  -nl
```
</div>

The result should resemble this plot:

<div>
<img src="images_xray_absorption_spectra_using_bse/graph2.jpg" width="500" align="left"/>
</div>

The plot can also be produced within this notebook by executing the following cell.

In [ ]:
plt.figure(figsize = (7, 5))

epsilon_results_TiO2: dict = parser_chooser(f'{cwd_TiO2}/EPSILON/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

plt.plot(epsilon_results_TiO2['frequency'], epsilon_results_TiO2['imag_oscillator_strength'],color = 'mediumblue')
plt.xlabel('Energy [eV]', fontsize = 17)
plt.ylabel(r"$\mathrm{Im}\;\; \varepsilon_M$", fontsize = 17)
plt.title('Macroscopic dielectric function', fontsize = 20)
plt.xlim(420, 470)
plt.ylim(0.000, 0.018)
plt.grid()

plt.show()

As all the main output files can be parsed by Python, in the example below, the results contained in main output file INFO.OUT are saved in the dictionary **converged_results_TiO2** and the results obtained for the loss function are saved in the dictionary **loss_results_TiO2**. The results obtained for the dielectric function are saved in the dictionary **epsilon_results_TiO2** as already defined above.

In [ ]:
results = parser_chooser(join(cwd_TiO2, 'INFO.OUT'))
max_scf = max([int(i) for i in results['scl'].keys()])
assert max_scf <= 30, "Expect max 30 SCF iterations to converge"
converged_results_TiO2: dict = results['scl'][str(max_scf)]
    
loss_results_TiO2: dict = parser_chooser(f'{cwd_TiO2}/LOSS/LOSS_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

We can further check that the obtained results are correct by comparing them with the reference calculations. To do so within the notebook, execute the following cell :

In [ ]:
from assertions_xray_absorption_bse import *

# Test results in main output file INFO.OUT 
test_groundstate_TiO2(converged_results_TiO2)

# Test results of dielectric function calculations
test_dielectric_TiO2(epsilon_results_TiO2, reference = 'reference_xray_absorption_bse/TiO2/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

# Test results of loss function calculations
test_loss_TiO2(loss_results_TiO2, reference = 'reference_xray_absorption_bse/TiO2/LOSS_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

**<span style="color:green">Exercise 2</span>**

- Perform a **RPA** calculation (<code><span style="color:mediumblue">bsetype</span>=<span style="color:firebrick">"RPA"</span></code>).
- Plot **RPA** absorption spectra, *e.g.*, using **EPSILON_BSE-RPA-TDA-BAR_SCR-full_OC11.OUT.xml** inside the **<span style="color:mediumblue">EPSILON</span>** subfolder and compare with the spectrum previously calculated from the full **BSE**. How do the spectra differ? Why?
- Search for bound core-exciton states in the file **EXCITON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT** inside the **<span style="color:mediumblue">EPSILON</span>** subfolder.
- Calculate the **L<sub>2</sub>** and **L<sub>3</sub>** subedges by setting the <code><span style="color:mediumblue">xasedge</span></code> attribute to **"L2"** and **"L3"**, respectively. Plot the different spectra in one plot. (**<span style="color:green">Suggestion</span>**: Copy the **<span style="color:mediumblue">EPSILON</span>** directory containing the **"L23"** results to **<span style="color:mediumblue">EPSILON-L23</span>**. Repeat the same procedure after the **"L2"** calculation to produce **<span style="color:mediumblue">EPSILON-L2</span>** and after the **"L3"** calculation to produce **<span style="color:mediumblue">EPSILON-L3</span>**). How do the three spectra differ? Why?


<hr style="border:1px solid #DDD"> </hr>

<strong><span style="color:firebrick">$\Rightarrow$ Solution: L2 and L3 spectra</span></strong>
 
<hr style="border:1px solid #DDD"> </hr>    

On terminal, after doing calculations, to plot all spectra in one plot, execute the following command in the working directory **TiO2-XANES**:


<div style="background-color: rgb(224, 224, 224);">
    
```bash
cp EPSILON-L23/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT singlet-L23
cp EPSILON-L2/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT singlet-L2
cp EPSILON-L3/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT singlet-L3
```
</div>

Then, run the script **PLOT-files.py** (for a detailed description of the script arguments see [<span style="color:firebrick">The python script "PLOT-files.py"</span>](http://exciting.wikidot.com/oxygen-the-python-script-plot-files-py)) with the following options:

<div style="background-color: rgb(224, 224, 224);">
    
```bash
PLOT-files.py -f singlet-L2 singlet-L23 singlet-L3  -lx 'Energy [eV]'  -ly 'Im $\varepsilon_M$'  -t 'Macroscopic dielectric function'  -x 420 470  -g  -rc  -cy 3  -lp 1
```
</div>

The resulting plot (**PLOT.png**) should look like this

<div>
<img src="images_xray_absorption_spectra_using_bse/graph3.jpg" width="500" align="left"/>
</div>

This can be done  within this notebook by executing the following cells.

In [ ]:
%%bash
cp -r TiO2-XANES/EPSILON TiO2-XANES/EPSILON-L23

In [ ]:
input_str = f"""
<input>
 
   <title>Rutile TiO2: Excited-State Calculation</title>
 
   <structure speciespath="{exciting_species}">
      <crystal>
         <basevect>   8.763555397       0.000000000       0.000000000</basevect>
         <basevect>   0.000000000       8.763555397       0.000000000</basevect>
         <basevect>   0.000000000       0.000000000       5.610015465</basevect>
      </crystal>
      <species speciesfile="Ti.xml" rmt="1.8000">
         <atom coord="0.0000000000      0.0000000000      0.0000000000"/>
         <atom coord="0.5000000000      0.5000000000      0.5000000000"/>
      </species>
      <species speciesfile="O.xml" rmt="1.8000">
         <atom coord="0.3050853616      0.3050853616      0.0000000000"/>
         <atom coord="0.6949146384      0.6949146384      0.0000000000"/>
         <atom coord="0.1949146384      0.8050853616      0.5000000000"/>
         <atom coord="0.8050853616      0.1949146384      0.5000000000"/>
      </species>
   </structure>
 
   <groundstate 
      do="skip"
      rgkmax="7.0"
      ngridk="4 4 6"
      xctype="GGA_PBE_SOL"
      epsocc="1.0d-6">
   </groundstate>
   
    <xs
      xstype="BSE" 
      ngridk="2 2 3"
      ngridq="2 2 3"
      rgkmax="7.0" 
      vkloff="0.05 0.15 0.25"
      nempty="30" 
      gqmax="1.0" 
      broad="0.018"
      tevout="true" >
 
      <energywindow 
         intv="15.0 18.0" 
         points="1500" />
      <screening 
         screentype="full" 
         nempty="75" />
      <BSE
         xas="true"
         xasspecies="1"
         xasatom="2"
         xasedge="L2" 
         bsetype="singlet"  
         nstlxas="1 20" />
      <qpointset>
         <qpoint>0.0 0.0 0.0</qpoint>
      </qpointset>
   </xs>
 
</input>"""

# Write out the input as an XML file:
with open(join(cwd_TiO2, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [ ]:
%%bash
# Move into the run directory
cd TiO2-XANES
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

In [ ]:
%%bash
cp -r TiO2-XANES/EPSILON TiO2-XANES/EPSILON-L2

In [ ]:
input_str = f"""
<input>
 
   <title>Rutile TiO2: Excited-State Calculation</title>
 
   <structure speciespath="{exciting_species}">
      <crystal>
         <basevect>   8.763555397       0.000000000       0.000000000</basevect>
         <basevect>   0.000000000       8.763555397       0.000000000</basevect>
         <basevect>   0.000000000       0.000000000       5.610015465</basevect>
      </crystal>
      <species speciesfile="Ti.xml" rmt="1.8000">
         <atom coord="0.0000000000      0.0000000000      0.0000000000"/>
         <atom coord="0.5000000000      0.5000000000      0.5000000000"/>
      </species>
      <species speciesfile="O.xml" rmt="1.8000">
         <atom coord="0.3050853616      0.3050853616      0.0000000000"/>
         <atom coord="0.6949146384      0.6949146384      0.0000000000"/>
         <atom coord="0.1949146384      0.8050853616      0.5000000000"/>
         <atom coord="0.8050853616      0.1949146384      0.5000000000"/>
      </species>
   </structure>
 
   <groundstate 
      do="skip"
      rgkmax="7.0"
      ngridk="4 4 6"
      xctype="GGA_PBE_SOL"
      epsocc="1.0d-6">
   </groundstate>
   
    <xs
      xstype="BSE" 
      ngridk="2 2 3"
      ngridq="2 2 3"
      rgkmax="7.0" 
      vkloff="0.05 0.15 0.25"
      nempty="30" 
      gqmax="1.0" 
      broad="0.018"
      tevout="true" >
 
      <energywindow 
         intv="15.0 18.0" 
         points="1500" />
      <screening 
         screentype="full" 
         nempty="75" />
      <BSE
         xas="true"
         xasspecies="1"
         xasatom="2"
         xasedge="L3" 
         bsetype="singlet"  
         nstlxas="1 20" />
      <qpointset>
         <qpoint>0.0 0.0 0.0</qpoint>
      </qpointset>
   </xs>
 
</input>"""

# Write out the input as an XML file:
with open(join(cwd_TiO2, 'input.xml'), "w") as fid:
    fid.write(input_str)

In [ ]:
%%bash
# Move into the run directory
cd TiO2-XANES
# Load the necessary modules
module load intel
# Run exciting
time $EXCITINGROOT/bin/exciting_smp input.xml
# Move back to parent directory 
cd ..

In [ ]:
%%bash
cp -r TiO2-XANES/EPSILON TiO2-XANES/EPSILON-L3

In [ ]:
from matplotlib.lines import Line2D

plt.figure(figsize = (7, 5))

epsilon_results_TiO2_L23: dict = parser_chooser(f'{cwd_TiO2}/EPSILON-L23/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')
epsilon_results_TiO2_L2: dict = parser_chooser(f'{cwd_TiO2}/EPSILON-L2/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')
epsilon_results_TiO2_L3: dict = parser_chooser(f'{cwd_TiO2}/EPSILON-L3/EPSILON_BSE-singlet-TDA-BAR_SCR-full_OC11.OUT')

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

plt.plot(epsilon_results_TiO2_L23['frequency'], epsilon_results_TiO2_L23['imag_oscillator_strength'],color = 'firebrick')
plt.plot(epsilon_results_TiO2_L2['frequency'], epsilon_results_TiO2_L2['imag_oscillator_strength'],color = 'green')
plt.plot(epsilon_results_TiO2_L3['frequency'], epsilon_results_TiO2_L3['imag_oscillator_strength'],color = 'mediumblue')
plt.xlabel('Energy [eV]', fontsize = 17)
plt.ylabel(r"$\mathrm{Im}\;\; \varepsilon_M$", fontsize = 17)
plt.title('Macroscopic dielectric function', fontsize = 20)
plt.xlim(420, 470)
plt.ylim(0.000, 0.018)
singlet_L23_label = Line2D( [0 ], [0 ], label = 'singlet-L23', color = 'firebrick')
singlet_L2_label = Line2D( [0 ], [0 ], label = 'singlet-L2', color = 'green')
singlet_L3_label = Line2D( [0 ], [0 ], label = 'singlet-L3', color = 'mediumblue')
plt.legend(handles = [singlet_L2_label, singlet_L23_label, singlet_L3_label])
plt.grid()

plt.show()